In [1]:
import random

from stox.imports import *
from stox.api import IB_Extractor
from stox import misc, yahoo
from omnibelt import tqdmd_notebook as tqdmd
from omnibelt import save_yaml
from collections import Counter
# from stox.alpha_vantage import AlphaVantageStocks

In [5]:
contracts = {}

In [40]:
root = misc.assets_root()
path = root / 'symbols.yml'
data = load_export(path)
existing = Counter(row['yahoo'] for row in data)
len(data), len(existing), existing.most_common(2)

(270, 270, [('MC.PA', 1), ('ASML.AS', 1)])

In [19]:
ibe = IB_Extractor(True)

ERROR:ib_insync.wrapper: Error 321, reqId 2147483647: Error validating request.-'cG' : cause - The API interface is currently in Read-Only mode.
ERROR:ib_insync.wrapper: Error 321, reqId -1: Error validating request.-'ce' : cause - The API interface is currently in Read-Only mode.
ERROR:ib_insync.ib: open orders request timed out
ERROR:ib_insync.ib: completed orders request timed out


In [38]:
bad = []
for row in tqdmd(data,key=lambda x: x['ibkr']):
	if row['ibkr'] not in contracts:
		contracts[row['ibkr']] = ibe.find_all_contracts(row['ibkr'], currency=row['currency'])
	cts = contracts[row['ibkr']]
	if len(cts) == 0 or len(set(c.conId for c in cts)) > 1:
		bad.append((row, cts))
		continue
	row['ibkr-ID'] = cts[0].conId
	row['ibkr-exchange'] = cts[0].primaryExchange
len(bad)

  0%|          | 0/271 [00:00<?, ?it/s]

6

In [63]:
# save_yaml(data, path);

In [39]:
print(tabulate([[row['ibkr'], row['yahoo'], len(cts)] for row, cts in bad], headers=['IBKR', 'Yahoo', 'Num']))

IBKR    Yahoo       Num
------  --------  -----
AIR     AIR.PA       31
AIR     AIR.DE       31
ENR     ENR.DE       16
ANA     ANA.MC       11
AGS     AGS.BR       16
HUH1V   HUH1V.HE      0


In [61]:
row, cts = bad[4]
print(row)

{'currency': 'EUR', 'ibkr': 'AGS', 'yahoo': 'AGS.BR', 'ibkr-ID': 111604420, 'ibkr-exchange': 'ENEXT.BE'}


In [62]:
for entry in data:
	if entry['ibkr'] == 'AGS' and entry['yahoo'] == 'AGS.MC':
		entry.update(row)
		break

In [50]:
row['ibkr-ID'] = cts[0].conId
row['ibkr-exchange'] = cts[0].primaryExchange

In [41]:
tkquery = 'AIR'
out = ibe.search(tkquery)
options = [c for c in out if c.secType == 'STK']
print(tabulate([(c.symbol, c.currency, c.description) for c in reversed(options)]))

---------  ---  ----------------------------
UAL        USD  UNITED AIRLINES HOLDINGS INC
600760     CNH  AVIC SHENYANG AIRCRAFT CO -A
LUV        USD  SOUTHWEST AIRLINES CO
DAL        USD  DELTA AIR LINES INC.
AIPUY      USD  AIRPORTS OF THAILAND PCL-ADR
NYVQ       EUR  AIRPORTS OF THAILAND PC-NVDR
TX3A       EUR  AIRPORTS OF THAILAND PCL-FOR
TATD       SGD  AIRPORTS OF THAILAND-SDR
ABNB       USD  AIRBNB INC-CLASS A
BHARTIART  INR  BHARTI AIRTEL LIMITED
APD        USD  AIR PRODUCTS & CHEMICALS INC
AIQUY      USD  AIR LIQUIDE-UNSPONSORED ADR
AI         EUR  AIR LIQUIDE SA
EADSY      USD  AIRBUS SE - UNSP ADR
LTMAY      USD  LATAM AIRLINES GROUP-SP ADR
AIROLAM    INR  AIRO LAM LTD
AIRTP      USD  AIR T FUNDING
AIRAN      INR  AIRAN LTD
AIRE       CHF  AIRESIS-REG
AIRG       USD  AIRGAIN INC
AIRX       NOK  AIRTHINGS ASA
AIRT       USD  AIR T INC
AIRE       GBP  ALTERNATIVE INCOME REIT PLC
AIRL       GBP  AIR PARTNER PLC
AIRS       USD  AIRSCULPT TECHNOLOGIES INC
AIRR       USD  FIRST TR

In [32]:
ibtk = tkquery.upper()
info = xmltodict.parse(ibe.snapshot(ibtk))
# company_name = info['ReportSnapshot']['CoIDs']['CoID'][1]['#text']
industries = [e['#text'] for e in info['ReportSnapshot']['peerInfo']['IndustryInfo']['Industry']]
print('\n'.join(industries))
print()
summary = info['ReportSnapshot']['TextInfo']['Text'][0]['#text']
print(summary)

Integrated Oil & Gas
Crude Petroleum Extraction
Natural Gas Extraction
Petroleum Refineries
Petroleum Lubricating Oil and Grease Manufacturing
Support Activities for Oil and Gas Operations
Pipeline Transportation of Natural Gas
Gasoline Stations with Convenience Stores
All Other Basic Organic Chemical Manufacturing
Other Electric Power Generation
Offices of Other Holding Companies
Crude Petroleum And Natural Gas
Crude Petroleum And Natural Gas
Petroleum Refining
Lubricating Oils And Greases
Oil And Gas Field Services, Nec
Natural Gas Transmission
Gasoline Service Stations
Industrial Organic Chemicals, Nec
Electric Services
Holding Companies, Nec

Shell plc (Shell) is an international energy and petrochemical company. The Company is engaged in the exploration, production, refining and marketing of oil and natural gas, and the manufacturing and marketing of chemicals. The Companys segments include Integrated Gas, Upstream, Marketing, Chemicals and Products, Renewables and Energy Solution

In [2]:
ibe.ib.disconnect()